In [1]:
import pandas as pd
import numpy as np

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic,  KNNWithMeans, KNNBaseline,KNNWithZScore
from surprise.model_selection import KFold
from surprise import SlopeOne, CoClustering
from surprise import SVD, SVDpp, NMF
from surprise import Reader
from surprise import NormalPredictor
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
user_reviews = pd.read_csv('.\\movie_reviews\\user_reviews.csv')
user_reviews = user_reviews.drop(user_reviews.columns[0], axis=1)
user_reviews.head()

,User,The Net,Happily N'Ever After,Tomorrowland,American Hero,Das Boot,Final Destination 3,Licence to Kill,The Hundred-Foot Journey,The Matrix,...,The Martian,Micmacs,Solomon and Sheba,In the Company of Men,Silent House,Big Fish,Get Real,Trading Places,DOA: Dead or Alive,Hey Arnold! The Movie
0,Vincent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Edgar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Addilyn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Marlee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Javier,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
user_reviews['userID'] = range (0,user_reviews.shape[0])

In [22]:
moviesNames = list(user_reviews.columns[2:-2])
moviesID = range (0, len(moviesNames))
moviesNamesID = pd.DataFrame(moviesID, columns = ['movieID'])
moviesNamesID['variable'] = moviesNames
moviesNamesID.head()

,movieID,variable
0,0,Happily N'Ever After
1,1,Tomorrowland
2,2,American Hero
3,3,Das Boot
4,4,Final Destination 3


In [23]:
movie_genres = pd.read_csv('movie_reviews\\movie_genres.csv')
movie_genres= movie_genres.drop(movie_genres.columns[0], axis=1)
movie_genres.head()

,movie_title,genre_action,genre_adventure,genre_animation,genre_biography,genre_comedy,genre_crime,genre_documentary,genre_drama,genre_family,...,genre_mystery,genre_news,genre_reality-tv,genre_romance,genre_sci-fi,genre_short,genre_sport,genre_thriller,genre_war,genre_western
0,The Net,1,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
1,Happily N'Ever After,0,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Tomorrowland,1,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
3,American Hero,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,Das Boot,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0


In [24]:
moviesProperties = pd.DataFrame((user_reviews != 0).sum(), columns = ['Ratings Count']).iloc[2:]
moviesProperties.head()

,Ratings Count
Happily N'Ever After,10
Tomorrowland,11
American Hero,10
Das Boot,11
Final Destination 3,9


In [25]:
userMovieList = list(user_reviews[user_reviews.userID == 0].any(0)[user_reviews[user_reviews.userID == 0].any(0).values == True].keys()[2:-1])
moviesRatings = user_reviews[user_reviews.userID == 0][list(userMovieList)].melt()
moviesRatings['userID'] = [0]* moviesRatings.shape[0]
for i in range (1,user_reviews.shape[0]):
    userMovieListTemp = list(user_reviews[user_reviews.userID == i].any(0)[user_reviews[user_reviews.userID == i].any(0).values == True].keys()[2:-1])
    moviesRatingsTemp = user_reviews[user_reviews.userID == i][list(userMovieListTemp)].melt()
    moviesRatingsTemp['userID'] = [i]* moviesRatingsTemp.shape[0]
    moviesRatings = pd.concat([moviesRatings,moviesRatingsTemp])

In [26]:
moviesRatings = moviesRatings.reset_index()
moviesRatings = moviesRatings[['userID','value','variable']]
moviesRatings = pd.merge(moviesRatings,user_reviews[['User','userID']],how="inner",
    on='userID' )

In [27]:
moviesRatings.head()

,userID,value,variable,User
0,0,3.0,Shattered,Vincent
1,0,3.0,Passchendaele,Vincent
2,0,3.0,Broken Arrow,Vincent
3,0,4.0,Songcatcher,Vincent
4,0,5.0,The Importance of Being Earnest,Vincent


In [28]:
moviesRatings.tail()

,userID,value,variable,User
15919,599,4.0,The Bank Job,Sarai
15920,599,4.0,The Stepfather,Sarai
15921,599,2.0,Good Kill,Sarai
15922,599,2.0,Sugar Hill,Sarai
15923,599,3.0,Waking Ned Devine,Sarai


In [29]:
data = pd.merge(
    moviesRatings,
    moviesNamesID,
    how="inner",
    on='variable')
data = data.rename(columns={'value': 'rating','variable': 'movieName','ID': 'movieID','User': 'userName'})
data.head()

,userID,rating,movieName,userName,movieID
0,0,3.0,Shattered,Vincent,140
1,17,5.0,Shattered,Emely,140
2,83,3.0,Shattered,Ava,140
3,179,4.0,Shattered,Scarlett,140
4,387,5.0,Shattered,Paisley,140


In [31]:
reader = Reader(rating_scale=(1, 5))

data = data[['userID','userName','rating','movieID','movieName']]
dataRead = Dataset.load_from_df(data[['userID', 'movieID', 'rating']], reader)

In [32]:
anti_set = dataRead.build_full_trainset().build_anti_testset()

In [36]:
movies = data[['movieID' , 'movieName']].drop_duplicates(['movieID' , 'movieName'])
users = data[['userID' , 'userName']].drop_duplicates(['userID' , 'userName'])

# Basic K-NN

In [37]:
kf = KFold(n_splits=3)
algo = KNNBasic()
best_algo = None
best_rmse = 1000.0
best_pred = None
for trainset, testset in kf.split(dataRead):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    if rmse < best_rmse:
        best_algo = algo
        best_pred = predictions

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4000
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4144
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3917


In [38]:
best_pred[:5]

[Prediction(uid=471, iid=1100, r_ui=4.0, est=3.0, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid=265, iid=911, r_ui=1.0, est=4.0, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=271, iid=1252, r_ui=3.0, est=1, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid=334, iid=375, r_ui=4.0, est=4.0, details={'actual_k': 2, 'was_impossible': False}),
 Prediction(uid=150, iid=292, r_ui=3.0, est=3.4301196871171427, details={'was_impossible': True, 'reason': 'Not enough neighbors.'})]

# K-NN with means

In [39]:
kf = KFold(n_splits=5)
sim_options = {'name':'cosine'}
algo = KNNWithMeans(sim_options = sim_options)
best_algo = None
best_rmse = 1000.0
best_pred = None
for trainset, testset in kf.split(dataRead):
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)
    # Compute and print Root Mean Squared Error
    rmse = accuracy.rmse(predictions, verbose=True)
    if rmse < best_rmse:
        best_algo = algo
        best_rmse= rmse
        best_pred = predictions
print(best_rmse)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3426
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3547
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3555
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3628
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.3626
1.342621632143853


In [44]:
pred_df = pd.DataFrame(best_pred).merge(data , left_on = ['uid', 'iid'], right_on = ['userID', 'movieID'])
pred_df[['uid', 'iid',  'userName', 'userID', 'movieName', 'movieID', 'est','rating']]

,uid,iid,userName,userID,movieName,movieID,est,rating
0,337,249,Isaiah,337,Teen Wolf Too,249,2.345938,3.0
1,497,1208,Jeremy,497,House of Flying Daggers,1208,4.269149,4.0
2,281,332,Trinity,281,Highway,332,1.759259,3.0
3,109,983,Lillian,109,Serenity,983,1.657738,1.0
4,462,442,Memphis,462,My Name Is Khan,442,3.586207,4.0
...,...,...,...,...,...,...,...,...
3179,155,652,Millie,155,"Dude, Where's My Car?",652,3.562500,1.0
3180,332,1593,Summer,332,Across the Universe,1593,5.000000,4.0
3181,3,1662,Marlee,3,Old Joy,1662,2.930263,4.0
3182,90,1815,Daleyza,90,I Am Number Four,1815,3.214286,5.0


In [45]:
anti_pre = best_algo.test(anti_set)
pred_df = pd.DataFrame(anti_pre).merge(movies , left_on = ['iid'], right_on = ['movieID'])
pred_df = pd.DataFrame(pred_df).merge(users , left_on = ['uid'], right_on = ['userID'])

In [46]:
pred_df.head()

,uid,iid,r_ui,est,details,movieID,movieName,userID,userName
0,0,120,3.435537,5.000000,"{'actual_k': 1, 'was_impossible': False}",120,Perfume: The Story of a Murderer,0,Vincent
1,0,146,3.435537,3.160579,"{'actual_k': 4, 'was_impossible': False}",146,The Lovely Bones,0,Vincent
2,0,162,3.435537,3.888393,"{'actual_k': 1, 'was_impossible': False}",162,Ordinary People,0,Vincent
3,0,167,3.435537,3.054705,"{'actual_k': 2, 'was_impossible': False}",167,Van Wilder: Party Liaison,0,Vincent
4,0,294,3.435537,4.091809,"{'actual_k': 2, 'was_impossible': False}",294,Queen of the Damned,0,Vincent


# finding Recommendations for a user

# Movies prediction for Vincent

In [47]:
Vincent = pred_df[(pred_df['est']>3.0)&(pred_df['userID']==0)].sort_values(by='est', ascending=False).iloc[:5]


In [48]:
Vincent = Vincent[ ~Vincent['movieName'].isin(list(user_reviews[user_reviews.userID == 0].any(0)[user_reviews[user_reviews.userID == 0].any(0).values == True].keys()[2:-1]))]


In [49]:
Vincent[(Vincent['est']>3.0)&(Vincent['userID']==0)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


0                        Perfume: The Story of a Murderer
1838                                             Waitress
188                                      The Devil's Tomb
189     Pirates of the Caribbean: The Curse of the Bla...
1476                                           The Smurfs
Name: movieName, dtype: object

In [50]:
Vincent[(Vincent['est']>3.0)&(Vincent['userID']==0)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


0                        Perfume: The Story of a Murderer
1838                                             Waitress
188                                      The Devil's Tomb
189     Pirates of the Caribbean: The Curse of the Bla...
1476                                           The Smurfs
Name: movieName, dtype: object

# Movies prediction for Edgar

In [51]:
Edgar = pred_df[(pred_df['est']>3.0)&(pred_df['userID']==1)].sort_values(by='est', ascending=False).iloc[:5]


In [52]:
Edgar = Edgar[ ~Edgar['movieName'].isin(list(user_reviews[user_reviews.userID == 0].any(0)[user_reviews[user_reviews.userID == 0].any(0).values == True].keys()[2:-1]))]


In [53]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==1)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


1176526               Inglourious Basterds
1177501                       Frozen River
1177495                    Lords of London
1176457                               Hero
1177477    Snow White and the Seven Dwarfs
Name: movieName, dtype: object

In [54]:
Edgar[(Edgar['est']>3.0)&(Edgar['userID']==1)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


1176526               Inglourious Basterds
1177501                       Frozen River
1177495                    Lords of London
1176457                               Hero
1177477    Snow White and the Seven Dwarfs
Name: movieName, dtype: object

# Movies prediction for Addilyn

In [ ]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==2)].sort_values(by='est', ascending=False).iloc[:5]


In [55]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==2)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


336702                 Face/Off
335957          Pieces of April
335927                Spotlight
337160    Sense and Sensibility
337167                 Deadpool
Name: movieName, dtype: object

# Movies prediction for Marlee

In [56]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==3)].sort_values(by='est', ascending=False).iloc[:5]


,uid,iid,r_ui,est,details,movieID,movieName,userID,userName
759672,3,1310,3.435537,5.0,"{'actual_k': 2, 'was_impossible': False}",1310,Rugrats in Paris: The Movie,3,Marlee
760116,3,1751,3.435537,5.0,"{'actual_k': 2, 'was_impossible': False}",1751,The Names of Love,3,Marlee
760406,3,379,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",379,Veer-Zaara,3,Marlee
760782,3,634,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",634,Phantasm II,3,Marlee
759819,3,1206,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",1206,Snow Angels,3,Marlee


In [57]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==3)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


759672    Rugrats in Paris: The Movie
760116              The Names of Love
760406                     Veer-Zaara
760782                    Phantasm II
759819                    Snow Angels
Name: movieName, dtype: object

# Movies prediction for Javier

In [58]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==4)].sort_values(by='est', ascending=False).iloc[:5]


,uid,iid,r_ui,est,details,movieID,movieName,userID,userName
1167248,4,606,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",606,Monster House,4,Javier
1166780,4,1136,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",1136,Night Watch,4,Javier
1166952,4,1923,3.435537,5.0,"{'actual_k': 2, 'was_impossible': False}",1923,Exit Wounds,4,Javier
1167879,4,169,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",169,Red Sky,4,Javier
1166875,4,1751,3.435537,5.0,"{'actual_k': 1, 'was_impossible': False}",1751,The Names of Love,4,Javier


In [59]:
pred_df[(pred_df['est']>3.0)&(pred_df['userID']==4)].sort_values(by='est', ascending=False).iloc[:5]['movieName']


1167248        Monster House
1166780          Night Watch
1166952          Exit Wounds
1167879              Red Sky
1166875    The Names of Love
Name: movieName, dtype: object